## Parallel Pipeline
    Proto-type the parallel pipeline -- from reading in the images to collecting the metrics. 
    Verify performance of pipeline, then deploy on Pleaides 

#### Import Python Libraries

In [9]:
import matplotlib.pyplot as plt
from time import time, sleep
import numpy as np
import pandas as pd
import os
%matplotlib inline

#### Spin up Local Cluster

In [46]:
# To make sure that we are not running several clusters on the same host, 
# let's try to shut down any running IPython cluster first
!ipcluster stop

2016-03-12 15:23:23.870 [IPClusterStop] Stopping cluster [pid=2482] with [signal=2]


In [2]:
# start cluster with 'n' number of worker nodes
! ipcluster start -n=3 --daemon

In [3]:
# import the ipython parallel computing client 
from ipyparallel import Client

In [4]:
client = Client()
print "There are currently {} nodes in the cluster".format(len(client))

There are currently 3 nodes in the cluster


In [5]:
rc = Client()
dview = rc[:]

#### Load Image Names

In [10]:
image_path = "/Users/Alexander/NASA/NASA_Sample_Data/Images/"
filelist = []
for i, filename in enumerate(os.listdir(image_path)):
    if i != 0:
        filelist.append(filename)

In [11]:
# change path for Pleiades job
image_path = "/Users/Alexander/NASA/NASA_Sample_Data/Images/"
trans_image_path = "/Users/Alexander/NASA/trans_image_data_singleDay/"
feature_results_path = "/Users/Alexander/NASA/feature_extraction_results_singleDay/"

In [12]:
file_name_list = []
for filename in os.listdir(image_path):
    if "ds_store" not in filename.lower():
        file_name_list.append(filename)

In [13]:
# process images that are sufficiently differnt from each other
#test_index = np.arange(1, 400, 50) # used for diverse image sub-smaple
test_index = range(0,3)     
print "getting 3 sample images..."
test_files = [ file_name_list[index] for index in test_index]

getting 3 sample images...


In [14]:
test_files

['fd_M_96m_01d.3925.0000.fits',
 'fd_M_96m_01d.3925.0001.fits',
 'fd_M_96m_01d.3925.0002.fits']

In [15]:
file_name = test_files

#### Ex. How to import and use modules in parallel

In [16]:
dview.execute('import test_parallel_2')
dview.execute('results = test_parallel_2.get_pid()')
dview["results"]

[2498, 2499, 2500]

In [33]:
dview.execute('import Cylindrical_Map_Transformation')
dview.execute('import sunspot_feature_extraction')
dview.execute('import extract_features_script')

<AsyncResult: execute>

In [18]:
dview.scatter('file_name', file_name)

<AsyncResult: scatter>

In [ ]:
image_path,trans_image_path,feature_results_path,

In [41]:
dview.push(dict(image_path = image_path,
               trans_image_path=trans_image_path,
               feature_results_path=feature_results_path))

<AsyncResult: _push>

In [36]:
dview.push?

In [ ]:
for i, file_ in enumerate(test_files):
start = time()
# include parameter values for tunning
extract_features_from_images(image_path,trans_image_path,feature_results_path, file_)
end = time()
print "TIME ELAPSED {0} mins, IMAGE DONE {1}".format((end - start)/60.0, i)

In [45]:
dview.execute('extract_features_script.extract_features_from_images(image_path,trans_image_path,feature_results_path, file_name)').get()

CompositeError: one or more exceptions from call to method: execute
[0:execute]: TypeError: cannot concatenate 'str' and 'list' objects
[1:execute]: TypeError: cannot concatenate 'str' and 'list' objects
[2:execute]: TypeError: cannot concatenate 'str' and 'list' objects

In [27]:
dview['results_img']

CompositeError: one or more exceptions from call to method: _pull
[Engine Exception]NameError: name 'results_img' is not defined
[Engine Exception]NameError: name 'results_img' is not defined
[Engine Exception]NameError: name 'results_img' is not defined

In [22]:
@dview.parallel()
def run_job(passed_img):
    return extract_features_scripts.extract_features_from_images(image_path,trans_image_path,feature_results_path, passed_img)

In [74]:
# @dview.parallel() # can be called on sequences and in parallel
# def get_count(dummy):
#     return dummy, test_parallel_2.get_pid()

In [23]:
results = run_job.map(test_files)

In [24]:
results.get()

CompositeError: one or more exceptions from call to method: run_job
[Engine Exception]NameError: global name 'extract_features_scripts' is not defined
[Engine Exception]NameError: global name 'extract_features_scripts' is not defined
[Engine Exception]NameError: global name 'extract_features_scripts' is not defined